We performed other modifications to the original code:

1. We added a parameter save_models_each, to use together with the argument save_models, in case we want to save some of the models but not all of them (e.g. each 3 models).
2. We made a file main_use.py as an alternative of the main.py file, for a simpler application of the model.

We will use the directories:

In [1]:
import os

dir_base = os.getcwd()
dir_edsrpytorch = os.path.join(dir_base,"EDSR-PyTorch")     # ./EDSR-PyTorch
dir_src = os.path.join(dir_edsrpytorch,"src")               # ./EDSR-PyTorch/src
dir_pretrain = os.path.join(dir_edsrpytorch, "pre-train")   # ./EDSR-PyTorch/pre-train

# 1. Getting the model ready

For more information about the following cells, we can check our previous notebooks.

## 1.1 Getting the repository

In [2]:
# Cloning the repository as instructed in https://github.com/sanghyun-son/EDSR-PyTorch
!git clone https://github.com/thstkdgus35/EDSR-PyTorch {dir_edsrpytorch}

Cloning into '/content/EDSR-PyTorch'...
remote: Enumerating objects: 806, done.
remote: Total 806 (delta 0), reused 0 (delta 0), pack-reused 806
Receiving objects: 100% (806/806), 63.09 MiB | 22.36 MiB/s, done.
Resolving deltas: 100% (516/516), done.


## 1.2 Get a pretrained model

In [4]:
import os
import urllib.request

# Pre-train folder
if not os.path.exists(dir_pretrain):
    os.makedirs(dir_pretrain, exist_ok=True)

# Pretrained model
pretrain_model = "edsr_x4-4f62e9ef.pt"
pretrain_model_path = os.path.join(dir_pretrain, pretrain_model)

# Download it if not present
if not os.path.isfile(pretrain_model_path):
  url = "https://cv.snu.ac.kr/research/EDSR/models/edsr_x4-4f62e9ef.pt"
  with urllib.request.urlopen(url) as response, open(pretrain_model_path, 'wb') as out_file:
    data = response.read()
    out_file.write(data)
  print(f"Pretrained model {pretrain_model} has been downloaded inside {dir_pretrain}")
else :
  print(f"Using pretrained model {pretrain_model_path}")

Pretrained model edsr_x4-4f62e9ef.pt has been downloaded inside /content/EDSR-PyTorch/pre-train


## 1.3 Move current working directory to the src folder

In [5]:
## Change the current working directory to EDSR-PyTorch/src, to run the model
import os

os.chdir(dir_src)
print(f"The Current working directory is: {os.getcwd()}")

The Current working directory is: /content/EDSR-PyTorch/src


We can change the current working directory back to the base directory at any time by uncommenting and running:

In [6]:
#os.chdir(dir_base)
#print(f"The Current working directory is: {os.getcwd()}")

## 1.4 Setting up the template.py file

In [7]:
# Setting up the template.py file
"""
NOTE: This will always try to read the template.py file from a file
named as template-backup.py, which corresponds to the original
template.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

# Lines to add to the file
template = [
    '    if args.template.find(\'EDSR_custom\') >= 0:\n',
    '        args.dir_data =  \"../image-data\"\n',
    '        args.data_train = \"Custom\"\n',
    '        args.data_test = \"Custom\"\n',
    '        args.data_range = \"1-2400/2401-2500\"\n',
    '        args.ext = \"sep\"\n',
    '        args.scale = \"4\"\n',
    '        args.model = \"EDSR\"\n',
    '        args.pre_train = \"../pre-train/edsr_x4-4f62e9ef.pt\"\n',
    '        args.n_resblocks = 32\n',
    '        args.n_feats = 256\n',
    '        args.res_scale = 0.1\n',
    '        args.test_every = 100\n',
    '        args.epochs = 11\n',
    '        args.batch_size = 16\n',
    '        args.save = \"edsr_x4-train\"\n'
]
#Note: spaces/tabulations are important.

# Specify the file path
template_file_path = os.path.join(dir_src,'template.py')

# Have a back-up of the original template.py
template_file_path_backup = os.path.join(dir_src,'template-backup.py')

# If the back-up file is there, read it, or make it if not
if os.path.isfile(template_file_path_backup):
  # Read the content of the back-up file
  with open(template_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(template_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(template_file_path_backup, 'w') as file:
      file.writelines(lines)

# Insert the new lines at the desired location (on line 54)
for idx,line in enumerate(template):
  lines.insert(54+idx, line)

# Write the modified content back to the file
with open(template_file_path, 'w') as file:
    file.writelines(lines)

print(f"Template successfully added to {template_file_path}")

Template successfully added to /content/EDSR-PyTorch/src/template.py


# 1. Argument save_models_each

The argument save_models_each = n will make that if the argument "save_models" is passed as True to the script, when the execution of the training will save the state dictionaries of the trained models, it will only save the state dictionary if its epoch is a multiple of n, instead of for each epoch.

## 1.1 Modifying the _ _ init _ _ file in the "model" folder

In order to achieve this, we need to modify the _ _ init _ _ file inside the "model" folder which is in the src folder of the repository. We will need two lines and modify other three. We can do it with the following:

In [9]:
"""
NOTE: This will always try to read the __init__.py file from a file
named as init-backup.py, which corresponds to the original
__init__.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

# Add these lines
new_line1 = '        self.save_models_each = args.save_models_each\n'
new_line2 = '            if (int(epoch) % int(self.save_models_each) == 0):\n'

# Modify these lines (add tab to the original ones)
lines_to_modify = [
    '                save_dirs.append(\n',
    '                    os.path.join(apath, \"model_{}.pt\".format(epoch))\n',
    '                )\n'
]
#Note: spaces/tabulations are important.

# Specify the file path
init_file_path = os.path.join(dir_src,'model', '__init__.py')             # Original
init_file_path_backup = os.path.join(dir_src,'model', 'init-backup.py')   # Back-up

# If the back-up file is there, read it, or make it if not
if os.path.isfile(init_file_path_backup):
  # Read the content of the back-up file
  with open(init_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(init_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(init_file_path_backup, 'w') as file:
      file.writelines(lines)


# Modify the lines
for idx,line in enumerate(lines_to_modify):
  lines[73+idx] = line

# Insert lines at the desired locations
lines.insert(73, new_line2)
lines.insert(32, new_line1)

# Write the modified content back to the file
with open(init_file_path, 'w') as file:
    file.writelines(lines)

## 1.2 Modifying the option.py file

Now we just need to add the save_models_each argument to the option.py file. We can do it with:

In [10]:
"""
NOTE: This will always try to read the option.py file from a file
named as option-backup.py, which corresponds to the original
option.py from the EDSR-PyTorch repository.

If such file is not found, the code below will assume it's a fresh
clone of the repository, and it will make it.
"""

import os

# Add line to save models each number of epochs instead of every epoch
savemodels_lines = [
    "parser.add_argument('--save_models_each', type=str, default='1',\n",
    "                    help='save models each this number of epochs')\n\n"
]

# File paths
option_file_path = os.path.join(dir_src,'option.py')                 # Original
option_file_path_backup = os.path.join(dir_src,'option-backup.py')   # Back-up

# If the back-up file is there, read it, or make it if not
if os.path.isfile(option_file_path_backup):
  # Read the content of the back-up file
  with open(option_file_path_backup, 'r') as file:
      lines = file.readlines()
else:
  # Read the content of the original file
  with open(option_file_path, 'r') as file:
      lines = file.readlines()
  # Write the content to the back-up file
  with open(option_file_path_backup, 'w') as file:
      file.writelines(lines)

# Insert the save models lines at the desired location (line 144)
for idx,line in enumerate(savemodels_lines):
  lines.insert(144+idx, line)

# Write the modified content back to the file
with open(option_file_path, 'w') as file:
    file.writelines(lines)

print(f"Option file successfully updated in {option_file_path}")

Option file successfully updated in /content/EDSR-PyTorch/src/option.py


## 1.3 Passing the argument to the model script

We can now just pass the argument to the command line to use on the terminal. If for example we want to save models for each 5 epochs, we can do it with:

In [ ]:
# Command to train the model (from the src folder)
!python main.py --template EDSR_custom --save_models --save_models_each 5 --chop

# 2. Script main_use.py

We prepared a main_use.py script as a simpler alternative of the main.py script. This file must be placed in the src folder of the repository.

We can perform a quick application of a trained model by providing the following arguments:

* dir_demo: Path to the folder containing the images we want to upscale (in either PNG or JPG format).
* pretrain_path: Path to the state dictionary file of the EDSR x4 model we want to use to upscale the images.

And we can run the following command on a terminal:

In [ ]:
# Command to train the model (from the src folder)
!python main_use.py --data_test Demo --dir_demo {dir_demo} --scale 4 --n_resblocks 32 --n_feats 256 --res_scale 0.1 --pre_train {pretrain_path} --test_only --save_results

We then can expect a folder named "experiment" inside the folder dir_demo, containing the Super-Resolution upscalings of the images inside of dir_demo.